In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import pickle
import image_embeddings
import scipy.stats

def load_data(file):
    
    print('loading file: ' + file)
    with open(file, 'rb') as f:
        data = pickle.load(f)

    return(data)

def find_cat(name_dict, cat):
    keep = []
    for key_id, key in enumerate(name_dict):  
        if cat in key:
            keep.append(key_id)
        
    return keep


In [3]:
dir(image_embeddings.cli)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [ ]:
exp_desc = pd.read_csv("exp_stimuli_desc.csv")

#--- index images
path_images = '/projects/crunchie/boyanova/EEG_Things/data_set/Images_all'
path_embeddings = '/projects/crunchie/boyanova/EEG_Things/data_set/embeddings'
[id_to_name, name_to_id, embeddings] = image_embeddings.knn.read_embeddings(path_embeddings)
index = image_embeddings.knn.build_index(embeddings)
topk = len(id_to_name)

data_record = {
              "image_name": [],
              "image_ID": [],
              "cond": [],
               "image_category":[],
              "image_path": []}


extension = '{}.jpg'
image_paths = exp_desc["stimuli_paths"].values
image_vector = [x.split("/")[-1][0:-4] for x in image_paths]

for ax_id, i in enumerate(image_vector):
    im_pt = os.path.join(path_images, extension.format(i))

    data_record["image_name"].append(extension.format(i))
    data_record["image_ID"].append(ax_id)
    data_record["image_path"].append(os.path.join(path_images, extension.format(i)))
    data_record["image_category"].append(i[0:-4])
    data_record["cond"].append("main")

    all_i_idx = find_cat(name_to_id, i[0:-3])
    alt_imgs = [id_to_name[x] for x in all_i_idx if id_to_name[x]]
    alt_emb = embeddings[all_i_idx]
    alt_index = image_embeddings.knn.build_index(alt_emb)
    alt_id_to_name = {key: value for key, value in id_to_name.items() if value in alt_imgs}
    alt_name_to_id = {key: name_to_id[key] for key in alt_imgs if key in name_to_id}
    new_keys = np.arange(0, len(alt_imgs))
    updated_dict = {new_keys[i]: value for i, (_, value) in enumerate(alt_id_to_name.items())}

    target_idx = find_cat(alt_name_to_id, i)

    alt_results = image_embeddings.knn.search(alt_index, updated_dict, alt_emb[target_idx][0], k = 3)
    alt_img = alt_results[1][1]

    im_pt = os.path.join(path_images, extension.format(alt_img))             
    data_record["image_name"].append(extension.format(alt_img))
    data_record["image_ID"].append(ax_id + 200)
    data_record["image_path"].append(os.path.join(path_images, extension.format(alt_img)))
    data_record["image_category"].append(alt_img[0:-4])
    data_record["cond"].append("alt")

In [ ]:
df = pd.DataFrame(data_record)
df.to_csv("/projects/crunchie/boyanova/EEG_Things/eeg_prep/exp_stimuli.csv")